In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.inspection import plot_partial_dependence

In [2]:
data = pd.read_csv("..//Data/compas-scores-two-years.csv")
display(data.columns)
#display(dada)


Index(['id', 'name', 'first', 'last', 'compas_screening_date', 'sex', 'dob',
       'age', 'age_cat', 'race', 'juv_fel_count', 'decile_score',
       'juv_misd_count', 'juv_other_count', 'priors_count',
       'days_b_screening_arrest', 'c_jail_in', 'c_jail_out', 'c_case_number',
       'c_offense_date', 'c_arrest_date', 'c_days_from_compas',
       'c_charge_degree', 'c_charge_desc', 'is_recid', 'r_case_number',
       'r_charge_degree', 'r_days_from_arrest', 'r_offense_date',
       'r_charge_desc', 'r_jail_in', 'r_jail_out', 'violent_recid',
       'is_violent_recid', 'vr_case_number', 'vr_charge_degree',
       'vr_offense_date', 'vr_charge_desc', 'type_of_assessment',
       'decile_score.1', 'score_text', 'screening_date',
       'v_type_of_assessment', 'v_decile_score', 'v_score_text',
       'v_screening_date', 'in_custody', 'out_custody', 'priors_count.1',
       'start', 'end', 'event', 'two_year_recid'],
      dtype='object')

In [7]:
#display(data["days_b_screening_arrest"])


In [10]:
data = data[data["days_b_screening_arrest"] <= 30]
data = data[data["days_b_screening_arrest"] >= -30]
df = data.drop(['dob', 'age_cat', 'v_screening_date','name','id', 'first', 'last', 'compas_screening_date','c_case_number','r_case_number', 'vr_case_number'],axis=1)
display(df)

,sex,age,race,juv_fel_count,decile_score,juv_misd_count,juv_other_count,priors_count,days_b_screening_arrest,c_jail_in,...,v_type_of_assessment,v_decile_score,v_score_text,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,Male,69,Other,0,1,0,0,0,-1.0,2013-08-13 06:03:42,...,Risk of Violence,1,Low,2014-07-07,2014-07-14,0,0,327,0,0
1,Male,34,African-American,0,3,0,0,0,-1.0,2013-01-26 03:45:27,...,Risk of Violence,1,Low,2013-01-26,2013-02-05,0,9,159,1,1
2,Male,24,African-American,0,4,0,1,4,-1.0,2013-04-13 04:58:34,...,Risk of Violence,3,Low,2013-06-16,2013-06-16,4,0,63,0,1
5,Male,44,Other,0,1,0,0,0,0.0,2013-11-30 04:50:18,...,Risk of Violence,1,Low,2013-11-30,2013-12-01,0,1,853,0,0
6,Male,41,Caucasian,0,6,0,0,14,-1.0,2014-02-18 05:08:24,...,Risk of Violence,2,Low,2014-03-31,2014-04-18,14,5,40,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,Male,23,African-American,0,7,0,0,0,-1.0,2013-11-22 05:18:27,...,Risk of Violence,5,Medium,2013-11-22,2013-11-24,0,1,860,0,0
7210,Male,23,African-American,0,3,0,0,0,-1.0,2014-01-31 07:13:54,...,Risk of Violence,5,Medium,2014-01-31,2014-02-02,0,1,790,0,0
7211,Male,57,Other,0,1,0,0,0,-1.0,2014-01-13 05:48:01,...,Risk of Violence,1,Low,2014-01-13,2014-01-14,0,0,808,0,0
7212,Female,33,African-American,0,2,0,0,3,-1.0,2014-03-08 08:06:02,...,Risk of Violence,2,Low,2014-03-08,2014-03-09,3,0,754,0,0


In [6]:
X = data[["age_cat", "race", "sex", "priors_count", "c_charge_degree", "two_year_recid"]]
categorical_cols = ["age_cat", "race", "sex", "c_charge_degree"]
X = pd.get_dummies(X, columns=categorical_cols, prefix=categorical_cols, prefix_sep='_')
X = X.drop(["age_cat_25 - 45", "race_Caucasian", "sex_Male", "c_charge_degree_F"], axis=1)
Y = data["score_text"] != "Low"
display(data)

,id,name,first,last,compas_screening_date,sex,dob,age,age_cat,race,...,v_decile_score,v_score_text,v_screening_date,in_custody,out_custody,priors_count.1,start,end,event,two_year_recid
0,1,miguel hernandez,miguel,hernandez,2013-08-14,Male,1947-04-18,69,Greater than 45,Other,...,1,Low,2013-08-14,2014-07-07,2014-07-14,0,0,327,0,0
1,3,kevon dixon,kevon,dixon,2013-01-27,Male,1982-01-22,34,25 - 45,African-American,...,1,Low,2013-01-27,2013-01-26,2013-02-05,0,9,159,1,1
2,4,ed philo,ed,philo,2013-04-14,Male,1991-05-14,24,Less than 25,African-American,...,3,Low,2013-04-14,2013-06-16,2013-06-16,4,0,63,0,1
5,7,marsha miles,marsha,miles,2013-11-30,Male,1971-08-22,44,25 - 45,Other,...,1,Low,2013-11-30,2013-11-30,2013-12-01,0,1,853,0,0
6,8,edward riddle,edward,riddle,2014-02-19,Male,1974-07-23,41,25 - 45,Caucasian,...,2,Low,2014-02-19,2014-03-31,2014-04-18,14,5,40,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7209,10996,steven butler,steven,butler,2013-11-23,Male,1992-07-17,23,Less than 25,African-American,...,5,Medium,2013-11-23,2013-11-22,2013-11-24,0,1,860,0,0
7210,10997,malcolm simmons,malcolm,simmons,2014-02-01,Male,1993-03-25,23,Less than 25,African-American,...,5,Medium,2014-02-01,2014-01-31,2014-02-02,0,1,790,0,0
7211,10999,winston gregory,winston,gregory,2014-01-14,Male,1958-10-01,57,Greater than 45,Other,...,1,Low,2014-01-14,2014-01-13,2014-01-14,0,0,808,0,0
7212,11000,farrah jean,farrah,jean,2014-03-09,Female,1982-11-17,33,25 - 45,African-American,...,2,Low,2014-03-09,2014-03-08,2014-03-09,3,0,754,0,0


In [ ]:
def sigmoid(z):     #hvis?           True                False 
    return np.where(z >= 0, 1 / (1 + np.exp(-z)), np.exp(z) / (1 + np.exp(z)))

import numpy as np
def logistic(x, beta0, beta1):
    return np.exp(beta0+beta1*x)/(1+np.exp(beta0+beta1*x))

In [ ]:
control = sigmoid(  )
racialBias = np.exp( coefs["race_African-American"] ) / (1 - control + (control * np.exp(coefs["race_African-American"])))